In [2]:
!pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.4 MB 3.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/9.4 MB 3.7 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/9.4 MB 3.4 MB/s eta 0:00:03
   ----------- ---------------------------- 2.6/9.4 MB 3.4 MB/s eta 0:00:02
   -------------- ------------------------- 3.4/9.4 MB 3.5 MB/s eta 0:00:02
   ----------------- ---------------------- 4.2/9.4 MB 3.5 MB/s eta 0:00:02
   --------------------- ------------------ 5.0/9.4 MB 3.6 MB/s eta 0:00:02
   ------------------------ --------------- 5.8/9.4 MB 3.6 MB/s eta 0:00:01
   ---------------------------- ----------- 6.6/9.4 MB 3.6 MB/s eta 0:00:01
   ------------------------------- -------- 7.3/9.4 MB 3.6 MB/s eta 0:00:01
   ---------------------------------- ----- 8.1/9.4 MB 3.6 MB/s eta 0:00:01
   -------------------------------------- - 8.9/9.4 MB 3.6 MB/s eta 0:00:01
   ----------------

In [4]:
!pip install webdriver-manager

In [5]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
base_url = "https://eg.hatla2ee.com/ar/car/page/{}"
data = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver2 = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

n = 150

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

for page in range(1, 410):
    url = base_url.format(page)
    driver.get(url)

    car_cards = driver.find_elements(By.CLASS_NAME, 'newCarListUnit_data_contain')


    for card in car_cards:
        datee = card.find_element(By.CLASS_NAME, 'otherData_Date').text
        header = card.find_element(By.CLASS_NAME, 'newCarListUnit_header')
        link = header.find_element(By.TAG_NAME, 'a')
        href = link.get_attribute('href')
        # driver2 = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver2.get(href)
        car_card = driver2.find_elements(By.CLASS_NAME, 'DescDataRow')
        car = {
            'Brand':None,
            'Model':None,
            'Price':None,
            'First_Use':None,
            'Mileage':None,
            'City':None,
            'Transmission':None,
            'Color':None,
            'Fuel':None,
            'Date':None
        }
        car['Date'] = datee
        # name = name_price.text
        # lst = name.split('\n')
        # car.append(lst[0])
        # car.append(lst[1])
        for index,card in enumerate(car_card):
            child_elements = card.find_elements(By.XPATH, ".//*")  # the dot '.' means relative to current element
            for child in child_elements:
                class_attr = child.get_attribute('class')
                if class_attr == 'DescDataItem':
                    name_price = driver2.find_element(By.CLASS_NAME, 'usedCarTitleWrap').text #
                    # print(name_price)
                    lst = name_price.split('\n')
                    car['Price'] = lst[1]
                    category = child.find_element(By.CLASS_NAME, 'DescDataSubTit').text
                    d = child.find_element(By.CLASS_NAME, 'DescDataVal').text
                    if category == 'الماركة':
                        car['Brand'] = d
                    elif category == 'الموديل':
                        car['Model'] = d
                    elif category == 'أول إستخدام':
                        car['First_Use'] = d
                    elif category == 'مدينة':
                        car['City'] = d
                    elif category == 'اللون':
                        car['Color'] = d
                    elif category == 'الوقود':
                        car['Fuel'] = d
                    elif category == 'كم':
                        car['Mileage'] = d
                    elif category == 'ناقل الحركة':
                        car['Transmission'] = d
        values = []
        for key, value in car.items():
            values.append(value)
        data.append(values)
        

    if page%10 == 0:
        df = pd.DataFrame(data)
        df.to_csv(f'first{page}_pages.csv')        
    print(f"Page {page} scapped!")

data = pd.DataFrame(data)
driver2.quit()
driver.quit()

In [21]:
data

,0,1,2,3,4,5,6,7,8,9
0,رينو || Renault,لوجان || Logan,"330,000 جنيه",2012,"263,000 كم",الجيزة,مانيوال,فضي,بنزين,date_range\n2025-03-25
1,نيسان || Nissan,قشقاي || Qashqai,"1,150,000 جنيه",2020,"105,000 كم",البحيرة,أتوماتيك,الأزرق الداكن,بنزين,date_range\n2025-03-25
2,بي ام دبليو || BMW,X3,"2,550,000 جنيه",2018,"79,000 كم",الإسكندرية,أتوماتيك,رمادي,بنزين,date_range\n2025-03-25
3,رينو || Renault,كادجار || Kadjar,"800,000 جنيه",2017,"200,000 كم",المهندسين,أتوماتيك,أبيض,بنزين,date_range\n2025-03-25
4,شيفروليه || Chevrolet,بيك اب || Pickup,"600,000 جنيه",2012,"560,000 كم",6 أكتوبر,مانيوال,احمر,ديزل,date_range\n2025-03-25
...,...,...,...,...,...,...,...,...,...,...
6154,نيسان || Nissan,سنترا || Sentra,"485,000 جنيه",2013,"133,000 كم",المقطم,أتوماتيك,بترولي,بنزين,date_range\n2025-02-18
6155,اوبل || Opel,كورسا || Corsa,"230,000 جنيه",2006,"177,000 كم",أسوان,مانيوال,فضي,بنزين,date_range\n2025-02-18
6156,سيتروين || Citroën,C5,"570,000 جنيه",2014,"190,000 كم",6 أكتوبر,أتوماتيك,موكا,بنزين,date_range\n2025-01-26
6157,سيتروين || Citroën,c4 بيكاسو || C4 Picasso,"480,000 جنيه",2013,"177,000 كم",6 أكتوبر,أتوماتيك,احمر غامق,بنزين,date_range\n2025-01-26


In [22]:
data.columns = ['brand', 'model', 'price', 'year', 'mileage', 'city', 'transmission_type', 'color', 'fuel', 'date']

In [23]:
data

,brand,model,price,year,mileage,city,transmission_type,color,fuel,date
0,رينو || Renault,لوجان || Logan,"330,000 جنيه",2012,"263,000 كم",الجيزة,مانيوال,فضي,بنزين,date_range\n2025-03-25
1,نيسان || Nissan,قشقاي || Qashqai,"1,150,000 جنيه",2020,"105,000 كم",البحيرة,أتوماتيك,الأزرق الداكن,بنزين,date_range\n2025-03-25
2,بي ام دبليو || BMW,X3,"2,550,000 جنيه",2018,"79,000 كم",الإسكندرية,أتوماتيك,رمادي,بنزين,date_range\n2025-03-25
3,رينو || Renault,كادجار || Kadjar,"800,000 جنيه",2017,"200,000 كم",المهندسين,أتوماتيك,أبيض,بنزين,date_range\n2025-03-25
4,شيفروليه || Chevrolet,بيك اب || Pickup,"600,000 جنيه",2012,"560,000 كم",6 أكتوبر,مانيوال,احمر,ديزل,date_range\n2025-03-25
...,...,...,...,...,...,...,...,...,...,...
6154,نيسان || Nissan,سنترا || Sentra,"485,000 جنيه",2013,"133,000 كم",المقطم,أتوماتيك,بترولي,بنزين,date_range\n2025-02-18
6155,اوبل || Opel,كورسا || Corsa,"230,000 جنيه",2006,"177,000 كم",أسوان,مانيوال,فضي,بنزين,date_range\n2025-02-18
6156,سيتروين || Citroën,C5,"570,000 جنيه",2014,"190,000 كم",6 أكتوبر,أتوماتيك,موكا,بنزين,date_range\n2025-01-26
6157,سيتروين || Citroën,c4 بيكاسو || C4 Picasso,"480,000 جنيه",2013,"177,000 كم",6 أكتوبر,أتوماتيك,احمر غامق,بنزين,date_range\n2025-01-26


In [ ]:
data.to_csv('raw_data.csv', index=False)

In [ ]:
data

,brand,model,price,year,mileage,city,transmission_type,color,fuel,date,Unnamed: 0
0,اودي || Audi,Q3,"2,250,000 جنيه",2021.0,"65,000 كم",6 أكتوبر,أتوماتيك,زيتوني,بنزين,date_range\r\n2025-04-21,NaN
1,مرسيدس || Mercedes,C 180,"2,650,000 جنيه",2022.0,"38,000 كم",6 أكتوبر,أتوماتيك,ذهبي,بنزين,date_range\r\n2025-04-21,NaN
2,تويوتا || Toyota,لاند كروزر || Land Cruiser,"9,500,000 جنيه",2024.0,0 كم,المهندسين,أتوماتيك,أبيض,بنزين,date_range\r\n2025-04-21,NaN
3,هيونداي || Hyundai,اكسنت RB || Accent RB,"705,000 جنيه",2019.0,"27,000 كم",القاهرة,أتوماتيك,سماوى,بنزين,date_range\r\n2025-04-21,NaN
4,كيا || Kia,سبورتاج || Sportage,"1,400,000 جنيه",2021.0,68 كم,الجيزة,أتوماتيك,فيراني,بنزين,date_range\r\n2025-04-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6154,نيسان || Nissan,سنترا || Sentra,"485,000 جنيه",2013.0,"133,000 كم",المقطم,أتوماتيك,بترولي,بنزين,date_range\n2025-02-18,6154.0
6155,اوبل || Opel,كورسا || Corsa,"230,000 جنيه",2006.0,"177,000 كم",أسوان,مانيوال,فضي,بنزين,date_range\n2025-02-18,6155.0
6156,سيتروين || Citroën,C5,"570,000 جنيه",2014.0,"190,000 كم",6 أكتوبر,أتوماتيك,موكا,بنزين,date_range\n2025-01-26,6156.0
6157,سيتروين || Citroën,c4 بيكاسو || C4 Picasso,"480,000 جنيه",2013.0,"177,000 كم",6 أكتوبر,أتوماتيك,احمر غامق,بنزين,date_range\n2025-01-26,6157.0
